In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

In [3]:
import numpy as np

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [12]:
from sklearn.model_selection import train_test_split

In [5]:
!pip install pyarrow

In [5]:
import sklearn

In [12]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    #df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [13]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
# With out filering based on duration

In [14]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [15]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,1.683333
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00,0.233333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00,13.300000


In [16]:
len(df_train),len(df_val)

(3066766, 2913955)

In [17]:
df_train.duration.describe(percentiles=[0.95,0.96,0.97,0.98,0.99])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
50%      1.151667e+01
95%      3.646667e+01
96%      3.945000e+01
97%      4.335000e+01
98%      4.873333e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [18]:
df_val.duration.describe(percentiles=[0.95,0.96,0.97,0.98,0.99])

count    2.913955e+06
mean     1.601591e+01
std      4.284210e+01
min     -4.361667e+01
50%      1.180000e+01
95%      3.721667e+01
96%      4.033333e+01
97%      4.435000e+01
98%      4.990000e+01
99%      5.886667e+01
max      7.053617e+03
Name: duration, dtype: float64

In [22]:
np.std(df_train['duration'])

42.5943442974141

In [6]:
def read_dataframe_i(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train_i = read_dataframe_i('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val_i = read_dataframe_i('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
#After filtering based on duration

In [25]:
len(df_train_i) / len(df_train)

0.9812202822125979

In [9]:
#categorical = ['PU_DO'] #['PULocationID', 'DOLocationID']
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts = df_train_i[categorical].to_dict(orient='records')
# X_train = dv.fit_transform(train_dicts)
X = dv.fit_transform(train_dicts)
print("Dimensionality of the feature matrix January: ", X.shape[1])

# val_dicts = df_val[categorical + numerical].to_dict(orient='records')
val_dicts = df_val_i[categorical].to_dict(orient='records')
# X_val = dv.fit_transform(val_dicts)
Y = dv.fit_transform(val_dicts)
#print("Dimensionality of the feature matrix February: ", X_val.shape[1])

Dimensionality of the feature matrix January:  515


In [10]:
# target = 'duration'
# y_train = df_train_i[target].values
# y_val = df_val_i[target].values
target = 'duration'
X_target = df_train_i[target].values
Y_target = df_val_i[target].values

In [13]:
X_train, X_test, y1_train, y1_test = train_test_split(X, X_target, test_size=0.2, random_state=42) # January

In [15]:
Y_train, Y_test, y2_train, y2_test = train_test_split(Y, Y_target, test_size=0.2, random_state=42) # February

In [16]:
lr = LinearRegression()
lr.fit(X_train, y1_train)

y_pred = lr.predict(X_train)

mean_squared_error(y1_train, y_pred, squared=False)
#January

7.648123663879893

In [17]:
lr = LinearRegression()
lr.fit(Y_train, y2_train)

y_pred = lr.predict(Y_train)

mean_squared_error(y2_train, y_pred, squared=False)
#February

7.774928830560207